## 1. Importing necessary libraries and pickle files

In [79]:
import pandas as pd
# libraries for text preprocessing
import re
import string
from bs4 import BeautifulSoup
!pip install contractions
import contractions
from contractions import fix
!pip install emoji
import emoji
# !pip install pyspellchecker
# from spellchecker import SpellChecker
# spell = SpellChecker()
# tokenization
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
# Stop word removal
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
!pip install distance
# import distance

!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

import pickle

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
# Mounting a drive for accessing glove model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing trained XGBoost model
pickle_in=open("/content/drive/MyDrive/Career/AI/NLP/QuoraQuestionPairSimilarity/XGBoost_Model.pickle","rb")
x_cfl=pickle.load(pickle_in)
pickle_in.close()

In [80]:
# Importing trained XGBoost model
pickle_in=open("/content/drive/MyDrive/Career/AI/NLP/QuoraQuestionPairSimilarity/df_test_with_results.pickle","rb")
df_test_with_results=pickle.load(pickle_in)
pickle_in.close()

## 2. Defining data cleaning and featurization functions

In [12]:
# Data cleaning
def clean_text(text):
    text = text.lower()  # Lowercase
    # Replace certain special characters
    text = text.replace('%', ' percent')
    text = text.replace('$', ' dollar ')
    text = text.replace('₹', ' rupee ')
    text = text.replace('€', ' euro ')
    text = text.replace('@', ' at ')

    # The pattern '[math]' appears around 900 times in the whole dataset.
    text = text.replace('[math]', '')

    # Replacing some numbers with string eqivalents (not perfect, can be done better to account for more cases)
    text = text.replace(',000,000,000 ', 'b ')
    text = text.replace(',000,000 ', 'm ')
    text = text.replace(',000 ', 'k ')
    text = re.sub(r'([0-9]+)000000000', r'\1b', text)
    text = re.sub(r'([0-9]+)000000', r'\1m', text)
    text = re.sub(r'([0-9]+)000', r'\1k', text)

    # text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove non word and non white space characters
    text = " ".join([contractions.fix(doc) for doc in text.split()]) # expanding text contractions
    text = " ".join([emoji.demojize(doc) for doc in text.split()]) # handling emojis
    # text = " ".join([spell.correction(doc) for doc in text.split()]) # spell corrections
    return text

In [13]:
# Featurization
# Featurization functions
# Common word feature
def common_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return len(w1 & w2)

# Total words feature
def total_words(row):
    w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return (len(w1) + len(w2))

# Advanced Features
def fetch_token_features(row):

    q1 = row['question1']
    q2 = row['question2']

    SAFE_DIV = 0.0001

    STOP_WORDS = stopwords.words("english")

    token_features = [0.0]*8

    # Converting the Sentence into Tokens:
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])

    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])

    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))

    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))

    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))


    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)

    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])

    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])

    return token_features

  # Fuzzy Features
def fetch_fuzzy_features(row):

    q1 = row['question1']
    q2 = row['question2']

    fuzzy_features = [0.0]*4

    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [52]:
# Cleaning and var creating from the functions defined
def var_creation(df):
  df['question1']=df['question1'].apply(clean_text)
  df['question2']=df['question2'].apply(clean_text)
  #common word
  df['word_common'] = df.apply(common_words, axis=1)
  # Total words
  df['word_total'] = df.apply(total_words, axis=1)
  df['word_share'] = round(df['word_common']/df['word_total'],2)
  #Token features
  token_features = df.apply(fetch_token_features, axis=1)
  df["cwc_min"]       = list(map(lambda x: x[0], token_features))
  df["cwc_max"]       = list(map(lambda x: x[1], token_features))
  df["ctc_min"]       = list(map(lambda x: x[4], token_features))
  df["ctc_max"]       = list(map(lambda x: x[5], token_features))
  df["last_word_eq"]  = list(map(lambda x: x[6], token_features))
  #fuzzy_features
  fuzzy_features = df.apply(fetch_fuzzy_features, axis=1)
  # Creating new feature columns for fuzzy features
  df['fuzz_ratio'] = list(map(lambda x: x[0], fuzzy_features))
  df['fuzz_partial_ratio'] = list(map(lambda x: x[1], fuzzy_features))
  df['token_sort_ratio'] = list(map(lambda x: x[2], fuzzy_features))
  df['token_set_ratio'] = list(map(lambda x: x[3], fuzzy_features))
  return(df)

## 3 Deployment verification through test data points

In [95]:
## Check for test cases
# Limiting the row numbers
df_test_with_results_lim=df_test_with_results.iloc[:5000]
# Variable creation
df_test_with_results_lim=var_creation(df_test_with_results_lim)
#Prediction
X_test=df_test_with_results_lim[['cwc_max', 'cwc_min', 'token_set_ratio', 'fuzz_ratio','token_sort_ratio', 'ctc_min', 'word_share', 'fuzz_partial_ratio','ctc_max', 'last_word_eq']]
ddf_test=pd.DataFrame({'proba_1_dep':x_cfl.predict_proba(X_test)[:,0],
                      'proba_2_dep':x_cfl.predict_proba(X_test)[:,1],
                      'Prediction_dep':x_cfl.predict(X_test)})

df_test_with_results_lim.reset_index(drop=True, inplace=True)
ddf_test.reset_index(drop=True, inplace=True)

df_test_with_results_lim=pd.concat([df_test_with_results_lim,ddf_test], axis=1, join='outer')
df_test_with_results_lim

,question1,question2,proba_1,proba_2,Prediction,Actual,word_common,word_total,word_share,cwc_min,...,ctc_min,ctc_max,last_word_eq,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio,proba_1_dep,proba_2_dep,Prediction_dep
0,why has the us kept making dividisons between ...,why does the us always make deep division betw...,0.266179,0.733821,1,1,9,27,0.33,0.571420,...,0.692302,0.642853,1,84,83,83,82,0.266179,0.733821,1
1,what is the significance of the cherubim and s...,what are the roles of the cherubim and seraphim,0.487496,0.512504,1,1,6,16,0.38,0.666644,...,0.666659,0.666659,1,80,75,74,87,0.487496,0.512504,1
2,you have got 24 hours to live how will you spe...,if you are told that you have only 24 hours to...,0.364071,0.635929,1,1,9,32,0.28,0.799984,...,0.562496,0.473682,0,61,71,73,75,0.364071,0.635929,1
3,why does europe have a voltage of 220 v japan ...,what is the household voltage in europe and am...,0.999931,0.000069,0,0,5,38,0.13,0.399992,...,0.499995,0.128205,0,30,46,38,61,0.999931,0.000069,0
4,how do i overcome depression and jealousy,how do i overcome depression,0.678870,0.321130,0,1,5,12,0.42,0.999950,...,0.999980,0.714276,0,81,100,81,100,0.678870,0.321130,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,how can i master coding,how do i master coding,0.291284,0.708716,1,1,4,10,0.40,0.999950,...,0.799984,0.799984,1,89,86,84,93,0.291284,0.708716,1
4996,who are you voting for in the 2016 us presiden...,who are you voting for and why,0.328808,0.671192,1,1,7,20,0.35,0.999900,...,0.999986,0.538457,1,61,87,61,100,0.328808,0.671192,1
4997,how many permutations can be made with letters...,how many 3letter words exist in english,0.999964,0.000036,0,0,2,24,0.08,0.199996,...,0.285710,0.117646,0,38,54,25,34,0.999964,0.000036,0
4998,how can i handle my life,how can we handle life well,0.632126,0.367874,0,1,4,12,0.33,0.999950,...,0.666656,0.666656,0,78,83,78,88,0.632126,0.367874,0


In [96]:
##Printing the results
print("Prediction probability 0 is not matching for {} samples out of total {} samples.".format(abs(df_test_with_results_lim['proba_1']-df_test_with_results_lim['proba_1_dep']).sum(),df_test_with_results_lim.shape[0]))
print("Prediction is not matching for {} samples out of total {} samples.".format(abs(df_test_with_results_lim['Prediction']-df_test_with_results_lim['Prediction_dep']).sum(),
                                                                                                 df_test_with_results_lim.shape[0]))

Prediction probability 0 is not matching for 0.0 samples out of total 5000 samples.
Prediction is not matching for 0 samples out of total 5000 samples.


In [99]:
df_test_with_results_lim[['question1', 'question2', 'proba_1', 'proba_2', 'Prediction', 'Actual','proba_1_dep', 'proba_2_dep', 'Prediction_dep']]

,question1,question2,proba_1,proba_2,Prediction,Actual,proba_1_dep,proba_2_dep,Prediction_dep
0,why has the us kept making dividisons between ...,why does the us always make deep division betw...,0.266179,0.733821,1,1,0.266179,0.733821,1
1,what is the significance of the cherubim and s...,what are the roles of the cherubim and seraphim,0.487496,0.512504,1,1,0.487496,0.512504,1
2,you have got 24 hours to live how will you spe...,if you are told that you have only 24 hours to...,0.364071,0.635929,1,1,0.364071,0.635929,1
3,why does europe have a voltage of 220 v japan ...,what is the household voltage in europe and am...,0.999931,0.000069,0,0,0.999931,0.000069,0
4,how do i overcome depression and jealousy,how do i overcome depression,0.678870,0.321130,0,1,0.678870,0.321130,0
...,...,...,...,...,...,...,...,...,...
4995,how can i master coding,how do i master coding,0.291284,0.708716,1,1,0.291284,0.708716,1
4996,who are you voting for in the 2016 us presiden...,who are you voting for and why,0.328808,0.671192,1,1,0.328808,0.671192,1
4997,how many permutations can be made with letters...,how many 3letter words exist in english,0.999964,0.000036,0,0,0.999964,0.000036,0
4998,how can i handle my life,how can we handle life well,0.632126,0.367874,0,1,0.632126,0.367874,0


## 4 Question pair similarity for user questions

In [100]:
def question_similarity(question1,question2):
  print('-'*50)
  print("Question 1 is - ",question1)
  print("Question 2 is - ",question2)
  print('-'*50)
  # creating dictionary
  q_dic={'question1':[question1],'question2':[question2]}
  # Creating dataframe
  df=pd.DataFrame(q_dic)
  #Processing and featurization
  df=var_creation(df)
  #Prediction
  X_test=df[['cwc_max', 'cwc_min', 'token_set_ratio', 'fuzz_ratio','token_sort_ratio', 'ctc_min', 'word_share', 'fuzz_partial_ratio','ctc_max', 'last_word_eq']]
  print('Duplicate prediction probability is {} and Prediction is {}'.format(round(x_cfl.predict_proba(X_test)[:,1][0],3),x_cfl.predict(X_test)[0]))

In [107]:
pd.set_option('display.max_colwidth', None)
df_test_with_results[df_test_with_results['Actual']==1].head(10)

,question1,question2,proba_1,proba_2,Prediction,Actual
402125,Why has the U.S. kept making dividisons between Philippines and her neighbouring countries?,Why does the U.S. always make deep division between Philippines and her neighbouring countries?,0.266179,0.733821,1,1
19347,What is the significance of the Cherubim and Seraphim?,What are the roles of the Cherubim and Seraphim?,0.487496,0.512504,1,1
117654,"You've got 24 hours to live, how will you spend your last day on Earth?",If you are told that you have only 24 hours to live how you will spend it? With whom?,0.364071,0.635929,1,1
173061,How do I overcome depression and jealousy?,How do I overcome depression?,0.678870,0.321130,0,1
320332,How should I start preparing for the CFA Level 1 exam?,What is the best way for preparing CFA Level - 1 Exam?,0.346945,0.653055,1,1
238888,How would be our life be like without Google?,What would life be like without Google?,0.128230,0.871770,1,1
126445,Is it possible to use gene therapy to make your DNA match someone else's?,Is it possible to use gene therapy to make the DNA in all of your cells match someone else's?,0.347119,0.652881,1,1
124465,What are the top 10 Skullcandy headphones?,What are the best Skullcandy headphones? Why?,0.595427,0.404573,0,1
182088,Can a girl get pregnant by eating sperm?,Can I Get Pregnant From Oral Sex?,0.496117,0.503883,1,1
97227,Do facts truly exist?,Do facts exist?,0.522475,0.477525,0,1


In [108]:
pd.set_option('display.max_colwidth', None)
df_test_with_results[df_test_with_results['Actual']==0].head(10)

,question1,question2,proba_1,proba_2,Prediction,Actual
162790,"Why does Europe have a voltage of 220 V, Japan has 100 V, and North America has 110 V? Is 220 V better, more dangerous, or more efficient? Why doesn't the world converge into a more unified voltage?",What's the household voltage in Europe and American? 220V?,0.999931,0.000069,0,0
206191,"As a 6'4"" 180lbs size 13 boots intermediate rider, what size snowboard should I get?",Snowboarding: What stomp pad should I get?,0.994636,0.005364,0,0
237011,How do you defend yourself from someone trying to mess with your mind?,What should I do when someone try to mess with my head?,0.526099,0.473901,0,0
183679,How can you jerk off?,Is it bad to jerk off?,0.542229,0.457771,0,0
175716,Is the Goldfield Hotel being renovated? How long will the renovations continue?,Do I need permits to do my retail shop renovation?,0.999993,0.000007,0,0
17461,Life is totally meaningless. We live. We die. So what's the point?,At what point in your life did you realize you were going to die soon? (from old age),0.917783,0.082217,0,0
342095,"Why is it when you bite the inside of our lip, you continue to accidentally bite the same spot?",How long does it take until I can eat with bite blockers?,0.999999,0.000001,0,0
177193,How do I fix the error code 0x80070422 in Windows 10 store?,Windows 10 Store is failing to install apps with error 0x80073CF6. What can I do to fix it?,0.279472,0.720528,1,0
2445,What is a safe way to lose 40 pounds in 5 weeks?,Is it safe to lose 40 pounds in 2 weeks?,0.405792,0.594208,1,0
49957,What are the rule changes for formula 1 2017 season?,What kind of discharge the army reserve can give you when they discharge you due to missing drills?,0.999996,0.000004,0,0


In [109]:
pd.set_option('display.max_colwidth', 50)

In [102]:
## Question pair similarity based on user input
question1 = input('Enter first question\n')
question2 = input('Enter second question\n')
question_similarity(question1,question2)
#Question 1 is -  What is the best way to learn white hat hacking?
# Question 2 is -  What is the best way to start learning hacking?

--------------------------------------------------
Question 1 is -  What is the best way to learn white hat hacking?
Question 2 is -  What is the best way to start learning hacking?
--------------------------------------------------
Duplicate prediction probability is 0.5329999923706055 and Prediction is 1


In [111]:
## Question pair similarity based on user input
question1 = input('Enter first question\n')
question2 = input('Enter second question\n')
question_similarity(question1,question2)
#Question 1 is -  What are the top 10 Skullcandy headphones?
# Question 2 is -  What are the best Skullcandy headphones? Why?

Enter first question
What are the top 10 Skullcandy headphones?
Enter second question
What are the best Skullcandy headphones? Why?
--------------------------------------------------
Question 1 is -  What are the top 10 Skullcandy headphones?
Question 2 is -  What are the best Skullcandy headphones? Why?
--------------------------------------------------
Duplicate prediction probability is 0.4050000011920929 and Prediction is 0


In [112]:
## Question pair similarity based on user input
question1 = input('Enter first question\n')
question2 = input('Enter second question\n')
question_similarity(question1,question2)

Enter first question
How many states are in India?
Enter second question
Does India has multiple states?
--------------------------------------------------
Question 1 is -  How many states are in India?
Question 2 is -  Does India has multiple states?
--------------------------------------------------
Duplicate prediction probability is 0.24300000071525574 and Prediction is 0


In [113]:
## Question pair similarity based on user input
question1 = input('Enter first question\n')
question2 = input('Enter second question\n')
question_similarity(question1,question2)

Enter first question
Is Maharashtra one of the state in India?
Enter second question
To which country does a Maharashtra state belongs?
--------------------------------------------------
Question 1 is -  Is Maharashtra one of the state in India?
Question 2 is -  To which country does a Maharashtra state belongs?
--------------------------------------------------
Duplicate prediction probability is 0.24500000476837158 and Prediction is 0


In [114]:
## Question pair similarity based on user input
question1 = input('Enter first question\n')
question2 = input('Enter second question\n')
question_similarity(question1,question2)

Enter first question
Is Maharashtra one of the state in India?
Enter second question
Is Maharashtra one of the state of any country?
--------------------------------------------------
Question 1 is -  Is Maharashtra one of the state in India?
Question 2 is -  Is Maharashtra one of the state of any country?
--------------------------------------------------
Duplicate prediction probability is 0.4519999921321869 and Prediction is 0


In [115]:
## Question pair similarity based on user input
question1 = input('Enter first question\n')
question2 = input('Enter second question\n')
question_similarity(question1,question2)

Enter first question
Is Maharashtra one of the state in India?
Enter second question
Is Maharashtra a state?
--------------------------------------------------
Question 1 is -  Is Maharashtra one of the state in India?
Question 2 is -  Is Maharashtra a state?
--------------------------------------------------
Duplicate prediction probability is 0.3370000123977661 and Prediction is 0
